In [1]:
# Reading the Merged operation data
df = spark.read.table("merged_operation")

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 3, Finished, Available, Finished)

# Creating date dimension table 

In [2]:
from pyspark.sql.types import *
from delta.tables import*

# # Define the schema for the dimdate_gold table
DeltaTable.createIfNotExists(spark) \
.tableName("dimdate_gold") \
.addColumn("Date", StringType()) \
.addColumn("DateID", LongType())\
.execute()

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 4, Finished, Available, Finished)

# Defining dimdate_gold table with no duplicate

In [3]:
from pyspark.sql.functions import col, dayofmonth, month, year, date_format

# # Create dataframe for dimDate_gold

dfdimDate_gold = df.dropDuplicates(["Date"]).select(col("Date"))
# # Display the first 10 rows of the dataframe to preview your data

display(dfdimDate_gold.head(10))

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2d1a042b-26a7-4a61-870a-06610622ee3d)

# Creating Date ID


In [4]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, coalesce, max, lit

dfdimDate_temp = spark.read.table("dimDate_gold")

MAXDateID = dfdimDate_temp.select(coalesce(max(col("DateID")),lit(0)).alias("MAXDateID")).first()[0]

dfdimDate_gold = dfdimDate_gold.join(dfdimDate_temp,(dfdimDate_gold.Date == dfdimDate_temp.Date), "left_anti")

dfdimDate_gold = dfdimDate_gold.withColumn("DateID",monotonically_increasing_id() + MAXDateID + 1)

# # Display the first 10 rows of the dataframe to preview your data

display(dfdimDate_gold.head(10))

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 6, Finished, Available, Finished)

PySparkValueError: [CANNOT_INFER_EMPTY_SCHEMA] Can not infer schema from empty dataset.

In [7]:
dfdimDate_gold.write.format("delta").mode("overwrite").saveAsTable("dimDate_gold")


StatementMeta(, d1a27b26-d9df-40d8-9b8f-a88e83fc74de, 9, Finished, Available, Finished)

# Creating fact table

In [5]:
from pyspark.sql.types import *
from delta.tables import *

DeltaTable.createIfNotExists(spark) \
.tableName("factoperation_gold") \
.addColumn("DateID", LongType())\
.addColumn("ID", LongType()) \
.addColumn("Operating_ratio_8", StringType()) \
.addColumn("Operating_profit_margin_9", StringType()) \
.addColumn("profit_margin_10", StringType()) \
.addColumn("Total_operating_revenue_scheduled_and_charter_services_x_1_000", StringType()) \
.addColumn("Total_scheduled_services_x_1_000", StringType()) \
.addColumn("Total_charter_services_x_1_000", StringType()) \
.addColumn("All_other_operating_revenue_x_1_000", StringType()) \
.addColumn("Total_operating_expenses_x_1_000", StringType()) \
.addColumn("Net_income_loss_x_1_000", StringType()) \
.addColumn("Average_number_of_employees_8", StringType()) \
.addColumn("Operating_revenue_per_employee", StringType()) \
.addColumn("Tonne-kilometres_flown_per_employee", StringType()) \
.addColumn("Number_of_carriers_included_8", StringType()) \
.addColumn("Passengers_x_1_000_number", StringType()) \
.addColumn("Goods_carried_x_1_000_kilograms", StringType()) \
.addColumn("Passenger-kilometres_x_1_000_Numbers", StringType()) \
.addColumn("Passenger_tonne-kilometres_x_1_000", StringType()) \
.addColumn("Goods_tonne-kilometres_x_1_000", StringType()) \
.addColumn("Total_tonne-kilometres_x_1_000", StringType()) \
.addColumn("Hours_flown_x_1_000", StringType()) \
.execute()

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 7, Finished, Available, Finished)

In [8]:
dffactoperation_gold = df.alias("df1") \
    .join(dfdimDate_temp.alias("df2"), col("df1.Date") == col("df2.Date"), "left")\
    .select(
        col("df2.DateID"),
        col("df1.Operating_ratio_8"),
        col("df1.Operating_profit_margin_9"),
        col("df1.Profit_margin_10"),
        col("df1.Total_operating_revenue_scheduled_and_charter_services_x_1_000"),
        col("df1.Total_scheduled_services_x_1_000"),
        col("df1.Total_charter_services_x_1_000"),
        col("df1.All_other_operating_revenue_x_1_000"),
        col("df1.Total_operating_expenses_x_1_000"),
        col("df1.Net_income_loss_x_1_000"),
        col("df1.Average_number_of_employees_8"),
        col("df1.Operating_revenue_per_employee"),
        col("df1.Tonne-kilometres_flown_per_employee"),
        col("df1.Number_of_carriers_included_8"),
        col("df1.Passengers_x_1_000_number"),
        col("df1.Goods_carried_x_1_000_kilograms"),
        col("df1.Passenger-kilometres_x_1_000_Numbers"),
        col("df1.Passenger_tonne-kilometres_x_1_000"),
        col("df1.Goods_tonne-kilometres_x_1_000"), 
        col("df1.Total_tonne-kilometres_x_1_000"),
        col("df1.Hours_flown_x_1_000"),
    ).orderBy(col("df1.Operating_ratio_8"), col("df2.DateID"))
display(dffactoperation_gold.head(10))

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ed31cc8f-3dde-4ac5-9790-6571dd31b194)

# Load the new fact table 


In [9]:
dffactoperation_gold.write.format("delta").mode("overwrite").saveAsTable("factOperation_gold")

StatementMeta(, ba7bd43a-3ab1-4b51-825b-51131b80dffe, 11, Finished, Available, Finished)

AnalysisException: [DELTA_FAILED_TO_MERGE_FIELDS] Failed to merge fields 'Operating_ratio_8' and 'Operating_ratio_8'